# Lesson 6c: Context Managers and Practical Examples

In this lesson, we'll cover:
1. Resource management using context managers (`with` statement)
2. Creating custom context managers
3. Putting it all together with practical examples

## Context Managers with the `with` Statement

The `with` statement in Python provides a convenient way to manage resources like files, connections, and locks. It ensures proper acquisition and release of resources.

### File Handling with `with`

The most common use of context managers is for file operations.

In [ ]:
# Without using with (not recommended)
file = open("example.txt", "w")
try:
    file.write("Hello, World!")
finally:
    file.close()  # Ensure the file is closed even if an exception occurs

In [ ]:
# Using with (recommended)
with open("example.txt", "w") as file:
    file.write("Hello, World!")
# File is automatically closed when the with block exits

In [ ]:
# Reading from a file with with
try:
    with open("example.txt", "r") as file:
        content = file.read()
        print(content)
except FileNotFoundError:
    print("The file was not found")

### Multiple Context Managers

You can use multiple context managers in a single `with` statement.

In [ ]:
try:
    with open("input.txt", "r") as input_file, open("output.txt", "w") as output_file:
        # Read from input file and write to output file
        content = input_file.read()
        output_file.write(content.upper())  # Convert to uppercase
    print("File content copied and converted to uppercase")
except FileNotFoundError:
    print("One of the files was not found")

### Creating Custom Context Managers

You can create your own context managers in two ways:
1. By defining a class with `__enter__` and `__exit__` methods
2. By using the `contextlib.contextmanager` decorator

In [ ]:
# Method 1: Class-based context manager
class Timer:
    def __enter__(self):
        import time

        self.start_time = time.time()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        import time

        self.end_time = time.time()
        self.execution_time = self.end_time - self.start_time
        print(f"Execution time: {self.execution_time:.6f} seconds")
        # Return False to let exceptions propagate, True to suppress them
        return False


# Using our custom context manager
with Timer() as timer:
    # Do something time-consuming
    total = 0
    for i in range(1000000):
        total += i

In [ ]:
# Method 2: Using contextlib.contextmanager decorator
from contextlib import contextmanager
import time


@contextmanager
def timer():
    start_time = time.time()
    try:
        # Yield control back to the with block
        yield
    finally:
        end_time = time.time()
        print(f"Execution time: {end_time - start_time:.6f} seconds")


# Using our decorator-based context manager
with timer():
    # Do something time-consuming
    result = sum(range(1000000))

## Putting It All Together: Practical Example

Let's combine what we've learned to create a practical example: a function that reads data from a file, processes it, and handles potential errors.

In [ ]:
import csv
import os


class DataProcessingError(Exception):
    """Base exception for data processing errors"""

    pass


class InvalidDataError(DataProcessingError):
    """Raised when data is invalid"""

    pass


def process_csv_data(file_path):
    """Process data from a CSV file and return the sum of numeric values in a specific column."""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file '{file_path}' does not exist")

    try:
        total = 0
        with open(file_path, "r", newline="") as csv_file:
            reader = csv.DictReader(csv_file)

            if "value" not in reader.fieldnames:
                raise InvalidDataError("CSV file must contain a 'value' column")

            for row in reader:
                try:
                    value = float(row["value"])
                    total += value
                except ValueError:
                    # Skip non-numeric values but log a warning
                    print(f"Warning: Non-numeric value '{row['value']}' found in row {reader.line_num}")

        return total
    except csv.Error as e:
        raise DataProcessingError(f"CSV parsing error: {str(e)}") from e


# Example usage
try:
    # Create a sample CSV file
    with open("sample_data.csv", "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["id", "name", "value"])
        writer.writerow(["1", "Item 1", "10.5"])
        writer.writerow(["2", "Item 2", "20.3"])
        writer.writerow(["3", "Item 3", "invalid"])
        writer.writerow(["4", "Item 4", "15.7"])

    # Process the CSV file
    result = process_csv_data("sample_data.csv")
    print(f"Total of numeric values: {result}")

except FileNotFoundError as e:
    print(f"Error: {e}")
except InvalidDataError as e:
    print(f"Error: {e}")
except DataProcessingError as e:
    print(f"Error processing data: {e}")
finally:
    # Clean up - remove the sample file
    if os.path.exists("sample_data.csv"):
        os.remove("sample_data.csv")
        print("Sample file removed")

## Practice Exercise: Custom Context Manager

Create a custom context manager that measures and displays the memory usage before and after executing a block of code.

In [ ]:
# Your solution here
# Hint: You can use the psutil module to measure memory usage

## Summary

In this lesson, we've covered:

1. **Context Managers**:
   - Using the `with` statement for resource management
   - Creating custom context managers using classes or decorators
   - Working with multiple context managers

2. **Practical Examples**:
   - Combining imports, exception handling, and context managers
   - Creating robust data processing functions

Context managers are a powerful Python feature that allow for clean, readable code that properly manages resources regardless of whether exceptions occur or not. They're essential for professional Python development.